In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os,sys
import random
import argparse
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
from keras.models import load_model


from train import build_PSPNet_model, train_val_generator, train_model

from sklearn.model_selection import train_test_split

train_im_list = os.listdir('./data/train_images/')
train,val = train_test_split(train_im_list,test_size=0.20)

from ImageDataGenerator import ImageGenerator

train_gen = ImageGenerator(image_dir='./data/train_images/',
                           anno_dir='./data/labels/',
                           mode='train')
val_gen = ImageGenerator(image_dir='./data/train_images/',
                           anno_dir='./data/labels/',
                           mode='valid')

input_layer = Input((input_shape[0],input_shape[1],num_channels))
resnet_block = ResNet(input_layer)
#model = build_PSPNet_model(input_shape=(128,128),
#                           num_channels=3,
#                           loss_function='mean_squared_error')

/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/f